In [1]:
import tensorflow as tf
import numpy as np

### 不使用tf model，創建一層RNN

In [2]:
n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons]) ,dtype=tf.float32)
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons]) ,dtype=tf.float32)
b  = tf.Variable(tf.zeros([1, n_neurons]) ,dtype=tf.float32)

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1,Wx) +b)

init = tf.global_variables_initializer()

In [3]:
X0_batch = np.array([
    [0,1,2],
    [3,4,5],
    [6,7,8],
    [9,0,1]
])

X1_batch = np.array([
    [9,8,7],
    [0,0,0],
    [6,5,4],
    [3,2,1]
])

In [4]:
with tf.Session() as sess:
    sess.run(init)
    Y0_val,Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})
    print(Y0_val)
    

[[ 0.97787815  0.9995493  -0.40546837 -0.99443    -0.912637  ]
 [ 0.99767697  1.          0.9992052  -0.9999998  -0.99970406]
 [ 0.9997582   1.          1.         -1.         -0.99999905]
 [-0.99999964  0.99964935  1.          0.9999905  -0.5382053 ]]


### 通過時間靜態展開

static_rnn

In [5]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(
    basic_cell, [X0,X1], dtype=tf.float32
)

Y0, Y1 = output_seqs

####  簡化流程

In [6]:
tf.reset_default_graph()

n_steps = 2 # 時間 t1, t2, ...

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(
    basic_cell,
    X_seqs, 
    dtype=tf.float32
)
init = tf.global_variables_initializer()
outputs = tf.transpose(tf.stack(output_seqs),perm=[1,0,2])


In [7]:
# (4,2,3)
X_batch = [
    #   t=0    t=1
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
]

In [8]:
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})
    print(outputs_val)

[[[-0.53172    -0.06129047  0.5259849   0.7461831   0.26974818]
  [-1.          0.9851603   0.99629563  0.96319175 -0.999999  ]]

 [[-0.9992508   0.6339809   0.9485337   0.95923936 -0.9688956 ]
  [-0.96497154  0.50815076  0.5328627  -0.6100401  -0.13661185]]

 [[-0.99999905  0.9150075   0.9955182   0.9940622  -0.999713  ]
  [-0.9999999   0.9769739   0.9818434   0.22416943 -0.9999065 ]]

 [[-0.99967194 -0.2238567  -0.8682402   0.07244248 -0.9998998 ]
  [-0.9947384   0.90954226  0.69278395 -0.82396156 -0.99886864]]]


### 動態展開

dynamic_rnn

In [9]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [10]:
seq_length = tf.placeholder(tf.int32, [None]) # 迭代長度

outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

In [11]:
X_batch = np.array([
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]], #這個batch的 t2 沒有資料， 補 0
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
])

seq_length_batch = np.array([2,1,2,2])
init = tf.global_variables_initializer()

In [12]:
with tf.Session() as sess:
    init.run()
    out_val, state_val = sess.run([outputs,states],feed_dict={X:X_batch,seq_length:seq_length_batch})
    print(state_val)

[[-0.81785506  0.9999991  -0.99973786 -0.99928516  0.85950166]
 [-0.7025531   0.99975246 -0.80191535 -0.9969385   0.26512143]
 [-0.32151115  0.9998992  -0.99353904 -0.97632664  0.7759264 ]
 [-0.8950131   0.9917103  -0.979825   -0.7942098  -0.38231385]]


### Training RNN

In [13]:
from tensorflow.contrib.layers import fully_connected

tf.reset_default_graph()

n_steps   =  28
n_inputs  =  28
n_neurons = 150
n_outputs =  10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) # 輸入
y = tf.placeholder(tf.int32  , [None])                    # labels

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = fully_connected(states, n_outputs, activation_fn=None) # 最後一層, output
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
                                                          labels=y, logits=logits)

loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [14]:
# 加載數據
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [15]:
n_epochs = 15
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1,n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test  = accuracy.eval(feed_dict={X: X_test,  y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.8933333 Test accuracy: 0.9106
1 Train accuracy: 0.96 Test accuracy: 0.9462
2 Train accuracy: 0.94 Test accuracy: 0.9592
3 Train accuracy: 0.9533333 Test accuracy: 0.9562
4 Train accuracy: 0.9866667 Test accuracy: 0.9692
5 Train accuracy: 0.97333336 Test accuracy: 0.9643
6 Train accuracy: 0.9866667 Test accuracy: 0.961
7 Train accuracy: 0.97333336 Test accuracy: 0.9724
8 Train accuracy: 0.9866667 Test accuracy: 0.9674
9 Train accuracy: 0.9866667 Test accuracy: 0.9649
10 Train accuracy: 0.99333334 Test accuracy: 0.9706
11 Train accuracy: 0.9866667 Test accuracy: 0.9694
12 Train accuracy: 0.97333336 Test accuracy: 0.9757
13 Train accuracy: 0.9866667 Test accuracy: 0.9753
14 Train accuracy: 0.99333334 Test accuracy: 0.973


### 預測時間序列

In [21]:
t_min, t_max = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

In [22]:
tf.reset_default_graph()

n_steps   = 20
n_inputs  = 1
n_neurons = 100
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs ])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
cell = tf.contrib.rnn.OutputProjectionWrapper(  # rnn 10個(nodes)輸出，連接全連接層 輸出一個預測值
    tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
    output_size=n_outputs
)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [23]:
learning_rate = 0.001

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [25]:
n_iterations = 5000
batch_size   = 50

with tf.Session() as sess:
    
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        if iteration%100 ==0:
            mse = loss.eval(feed_dict={X:X_batch, y:y_batch})
            print(iteration, "\tMSE:",mse)

0 	MSE: 14.295551
100 	MSE: 0.45873547
200 	MSE: 0.16326381
300 	MSE: 0.07136521
400 	MSE: 0.053784274
500 	MSE: 0.04462791
600 	MSE: 0.05512355
700 	MSE: 0.049949203
800 	MSE: 0.049465798
900 	MSE: 0.043746725
1000 	MSE: 0.040598463
1100 	MSE: 0.04284016
1200 	MSE: 0.042445976
1300 	MSE: 0.04459831
1400 	MSE: 0.043287806
1500 	MSE: 0.045241427
1600 	MSE: 0.038770054
1700 	MSE: 0.04275372
1800 	MSE: 0.045583777
1900 	MSE: 0.043035455
2000 	MSE: 0.035891056
2100 	MSE: 0.04345268
2200 	MSE: 0.04104268
2300 	MSE: 0.05298567
2400 	MSE: 0.040818557
2500 	MSE: 0.03683746
2600 	MSE: 0.033581905
2700 	MSE: 0.041730445
2800 	MSE: 0.038496666
2900 	MSE: 0.033180237
3000 	MSE: 0.041560296
3100 	MSE: 0.042349033
3200 	MSE: 0.03794605
3300 	MSE: 0.03509926
3400 	MSE: 0.035193916
3500 	MSE: 0.03438435
3600 	MSE: 0.036999628
3700 	MSE: 0.041265767
3800 	MSE: 0.03916671
3900 	MSE: 0.040533196
4000 	MSE: 0.045616135
4100 	MSE: 0.038390335
4200 	MSE: 0.033646803
4300 	MSE: 0.033181913
4400 	MSE: 0.04478

NameError: name 't_instance' is not defined